<a href="https://colab.research.google.com/github/Espanta/handson-ml/blob/master/People_Satisfaction_and_GDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict People Satisfaction Across the Globe

Problem Statement:

We would like to build a model that predicts satisfaction score for people of different countries given their country GDP.

# Download Dataset

Download the Better Life Index data (latest edition, currently it is 2017) from the [OECD’s website](http://homl.info/4) as well as stats about GDP per capita from the [IMF’s website](http://homl.info/5). Then you join the tables and sort by GDP per capita. Table 1-1 shows an excerpt of what you get

# Import Dataset to Google Colab

1. Download CSV and XLS files to your computer
2. Upload them to your Google Drive
3. Open the CSV files using Google Sheets so Google will create the dataset in format of Google Sheets
4. You can remove CSV and XLS files from your drive
5. Use the step by step guide from [here](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=vz-jH8T_Uk2c) and scroll down to **" Google Sheets" ** cell to import data into dataframe

NOTE: After creating Google Sheet into your Drive, make sure you are converting Column 2015 to 0.00 format before importing it into Colab  otherwise Google will import it as a string and you will have hard time to clean the data




In [0]:
# Run below line of code for the first time to install gspread. Once installed comment it for future use
#!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

# Use gc to open Google Sheet Datasets

In [0]:
#Open given sheet
worksheet = gc.open('BLI_30012019054825599').sheet1

# Read contents of CSV file
bli_rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
bli  = pd.DataFrame.from_records(bli_rows, columns = bli_rows[0])

# Remove rows where inequality has values other than TOT
bli = bli[bli["INEQUALITY"]=="TOT"]

# Reformat data based on "indicator column"
bli = bli.pivot(index="Country", columns="Indicator", values="Value")

bli.head()

Indicator,Air pollution,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Feeling safe walking alone at night,Homicide rate,Household net adjusted disposable income,Household net financial wealth,Housing expenditure,...,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Stakeholder engagement for developing regulations,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education
Country,,,,,,,,,,,,,,,,,,,,,
Australia,5,1.1,80,13.2,72,63.6,1,33417,57462,20,...,52063,94,2.3,85,2.7,502,14.35,91,92,21.2
Austria,16,1,85,6.78,72,80.7,0.4,32544,59574,21,...,48295,92,1.6,70,1.3,492,14.55,75,93,17.1
Belgium,15,2.3,75,4.31,62,70.7,1,29968,104084,21,...,49587,92,2.2,75,2.2,503,15.77,89,84,18.2
Brazil,10,6.7,49,7.15,64,37.3,27.6,12227,7102,20,...,14024,90,0.8,70,2.2,395,14.45,79,72,15.9
Canada,7,0.2,91,3.73,73,80.9,1.4,29850,85758,22,...,48403,93,2.5,88,3,523,14.41,68,91,16.7


# Import WOE data

In [0]:
#Open given sheet
worksheet = gc.open('WEO_Data').sheet1

# Read contents of CSV file
WEO_rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
weo  = pd.DataFrame.from_records(WEO_rows, columns = WEO_rows[0])

# Drop the header row from data
weo = weo.reindex(weo.index.drop(0))

# 1- Select only Country name and 2015 
# 2- then rename it to GDP Per capita
weo = weo[['Country','2015']].rename(columns={'2015':'GDP per capita'})

# Set Country as index column
# Inplace command, will replace the results of command into the same DF
weo.set_index("Country", inplace=True)

#weo.drop_duplicates(inplace=True)
#Print top 5 rows
weo.head()

,GDP per capita
Country,
Afghanistan,599.99
Albania,3995.38
Algeria,4318.14
Angola,4100.32
Antigua and Barbuda,14414.30


In [0]:
bli["Life satisfaction"].head()

Country
Australia    7.3
Austria        7
Belgium      6.9
Brazil       6.6
Canada       7.3
Name: Life satisfaction, dtype: object

# Merge/Join dataset

In [0]:
df = pd.merge(left = weo, right = bli,   left_index=True, right_index=True)
df.sort_values(by="GDP per capita", inplace=True)
df.head()

,GDP per capita,Air pollution,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Feeling safe walking alone at night,Homicide rate,Household net adjusted disposable income,Household net financial wealth,...,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Stakeholder engagement for developing regulations,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education
Country,,,,,,,,,,,,,,,,,,,,,
Luxembourg,101994.09,12,0,79,3.76,66,72,0.6,41317,74141,...,62636,92,2,70,1.5,483,15.15,91,85,15.1
Hungary,12239.89,19,4.3,83,3.05,67,50.7,1.2,16821,23289,...,21711,84,1.2,56,1.2,474,15.06,62,76,16.6
Poland,12495.33,22,2.7,91,6.68,65,66.3,0.8,18906,14997,...,25921,89,1.1,58,2.6,504,14.42,55,80,17.7
Chile,13340.91,16,9.4,65,10.06,62,51.1,4.5,16588,21409,...,28434,84,1.9,57,1.5,443,14.9,49,69,17.3
Latvia,13618.57,11,12.9,89,2.09,69,60.7,6.6,15269,17105,...,22389,86,1.2,46,2.4,487,13.83,59,77,17.9


In [0]:
test_indices = [0, 1, 6, 8, 33, 34, 35]
train_indices = list(set(range(36)) - set(test_indices))

train = df[["GDP per capita", 'Life satisfaction']].iloc[train_indices]
test = df[["GDP per capita", 'Life satisfaction']].iloc[test_indices]



In [0]:
test

,GDP per capita,Life satisfaction
Country,,
Luxembourg,101994.09,6.9
Hungary,12239.89,5.3
Czech Republic,17256.92,6.6
Greece,18064.29,5.2
Switzerland,80675.31,7.5
Brazil,8670.00,6.6
Mexico,9009.28,6.6


In [0]:
# Code example
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression

# Prepare the data
X = np.c_[train["GDP per capita"]]
y = np.c_[train["Life satisfaction"]]

# Visualize the data
#df.plot(kind='scatter', x="GDP per capita", y='Life satisfaction')
#plt.show()

# Select a linear model
model = sklearn.linear_model.LinearRegression()

# Train the model
model.fit(X, y)

# Make a prediction for Cyprus
X_new = [[18064.29]]  # Cyprus' GDP per capita
print(model.predict(X_new)) # outputs [[5.95199478]]

[[5.95199478]]


In [0]:
# Make a prediction for our test data
model.predict(test['GDP per capita'].values.reshape(-1,1))

array([[9.16992716],
       [5.72868285],
       [5.9210396 ],
       [5.95199478],
       [8.35254892],
       [5.59181055],
       [5.60481881]])